Process Budzik et al data

3 Excel Worksheets: Abundance, Phospho, UB

4 timepoints

For each timepoint, merge and process the sheets to create a single spreadsheet to be loaded to Cytoscape so that we can build a model for each timepoint. Phospho and UB sites are summarized for a protein. Hence a protein can potentially be both up and down for PTMs.

Columns for the output spreadsheets:
- gene: Human Gene Symbol
- mouse_gene: Original Mouse Gene Symbol
- timepoint:
- ab_sig: Significant differential abundance
- ab_log2fc
- ab_adj_pval
- ab_up
- ab_down
- ph_sig
- ph_max_log2fc
- ph_max_adj_pval
- ph_up
- ph_up_sites: JSON of pval and fc for sites
- ph_down
- ph_down_sites: JSON of pval and fc of sites
- ub_sig
- ub_max_log2fc
- ub_max_adj_pval
- ub_up
- ub_up_sites: JSON of pval and fc for sites
- ub_down
- ub_down_sites: JSON of pval and fc of sites
- max_log2fc

In [1]:
# imports
from ndex2.nice_cx_network import NiceCXNetwork
import ndex2.client as nc
import ndex2
print("ndex2 version: " + ndex2.__version__)
import networkx as nx
print("networkx version: " + nx.__version__)
import pandas as pd
print("pandas version: " + pd.__version__)
import os
import sys
from gprofiler import GProfiler
gp = GProfiler(
    user_agent='hierarchical model analysis', #optional user agent
    return_dataframe=True, #return pandas dataframe or plain python structures    
    )

import plotly.graph_objects as go

ndex2 version: 3.3.1
networkx version: 2.4
pandas version: 1.0.3


In [14]:
# loading
#os.getcwd()
#files = os.listdir()
#file = files[1]
#files
#raw_abundance = pd.read_excel(file)

timepoint = '24H'
ab_file = 'budzik 2020 Mtb Protein Abundance.xlsx'
ph_file = 'budzik 2020 Mtb Phosporylation.xlsx'
ub_file = 'budzik 2020 Mtb Ubiquitylation.xlsx'
raw_ab_df = pd.read_excel(ab_file, sheet_name=timepoint)
raw_ph_df = pd.read_excel(ph_file, sheet_name=timepoint)
raw_ub_df = pd.read_excel(ub_file, sheet_name=timepoint)
raw_ub_df

,Protein,Gene,Entry.name,Uniprot_PTM,Comparison,log2FC,pvalue,adj.pvalue,imputed,iLog2FC,iPvalue,Protein.names
0,A1L314,Mpeg1,MPEG1_MOUSE,A1L314_K142,TB_24H-MOCK_24H,NaN,NaN,0.000000,yes,4.755452,0.028939,Macrophage-expressed gene 1 protein (Macrophag...
1,A1L314,Mpeg1,MPEG1_MOUSE,A1L314_K128,TB_24H-MOCK_24H,NaN,NaN,0.000000,yes,3.589451,0.014883,Macrophage-expressed gene 1 protein (Macrophag...
2,A2ADY9,Ddi2,DDI2_MOUSE,A2ADY9_K283,TB_24H-MOCK_24H,-0.055181,0.820814,0.895638,no,-0.055181,0.895638,Protein DDI1 homolog 2
3,A2ADY9,Ddi2,DDI2_MOUSE,A2ADY9_K289,TB_24H-MOCK_24H,-0.398821,0.091370,0.214544,no,-0.398821,0.214544,Protein DDI1 homolog 2
4,A2AGT5,Ckap5,CKAP5_MOUSE,A2AGT5_K96,TB_24H-MOCK_24H,1.644335,0.002904,0.015994,no,1.644335,0.015994,Cytoskeleton-associated protein 5
...,...,...,...,...,...,...,...,...,...,...,...,...
1801,Q9Z2U0,Psma7,PSA7_MOUSE,Q9Z2U0_K38,TB_24H-MOCK_24H,1.764534,0.000667,0.004617,no,1.764534,0.004617,Proteasome subunit alpha type-7 (EC 3.4.25.1) ...
1802,Q9Z2U1,Psma5,PSA5_MOUSE,Q9Z2U1_K196,TB_24H-MOCK_24H,1.289431,0.000012,0.000149,no,1.289431,0.000149,Proteasome subunit alpha type-5 (EC 3.4.25.1) ...
1803,Q9Z2U1,Psma5,PSA5_MOUSE,Q9Z2U1_K192,TB_24H-MOCK_24H,0.776024,0.081200,0.196623,no,0.776024,0.196623,Proteasome subunit alpha type-5 (EC 3.4.25.1) ...
1804,Q9Z2U1,Psma5,PSA5_MOUSE,Q9Z2U1_K187,TB_24H-MOCK_24H,1.067957,0.015412,0.057463,no,1.067957,0.057463,Proteasome subunit alpha type-5 (EC 3.4.25.1) ...


In [15]:
# timepoint processing functions

# get the human gene symbol

# make abundance DF
#
# comparison
# ab_sig: Significant differential abundance
# ab_log2fc
# ab_adj_pval
# ab_up
# ab_down

sig_pval = 0.1
sig_log2FC = 0.5
def is_sig_change(row):
    if row['iLog2FC'] > sig_log2FC and row['adj.pvalue'] < sig_pval:
        return 1
    return 0

def is_sig_up(row):
    if row['iLog2FC'] > sig_log2FC and row['adj.pvalue'] < sig_pval:
        return 1
    return 0

def is_sig_down(row):
    if row['iLog2FC'] < (-1.0 * sig_log2FC) and row['adj.pvalue'] < sig_pval:
        return 1
    return 0

def make_ab_df(raw_df):
    ab_df = raw_df[['Gene', 'Protein', 'iLog2FC', 'iPvalue']]
    ab_df = ab_df.rename(columns={'Gene': 'mouse_gene', 
                                                'Protein': 'protein', 
                                                'iLog2FC': 'ab_ilog2fc',
                                                'iPvalue': 'ab_ipvalue'
                                               })
    ab_df['name'] = ab_df['mouse_gene'].str.upper()
    # significant decrease
    ab_df['ab_sig_down'] = raw_df.apply(lambda x: is_sig_down(x), axis=1)
    # significant decrease
    ab_df['ab_sig_up'] = raw_df.apply(lambda x: is_sig_up(x), axis=1)
    # convenience column to iden
    ab_df['ab_detected'] = 1
    #ab_df = ab_df.set_index('protein')
    return ab_df

def make_ph_df(raw_df):
    ph_df = raw_df[['Gene', 'Protein','iLog2FC', 'iPvalue', 'Uniprot_PTM']]
    ph_df = ph_df.rename(columns={'Gene': 'mouse_gene', 
                                                'Protein': 'protein', 
                                                'iLog2FC': 'ph_ilog2fc',
                                                'iPvalue': 'ph_ipvalue',
                                                'Uniprot_PTM': 'ph_ptm'
                                               })
    ph_df['name'] = ph_df['mouse_gene'].str.upper()
    # significant decrease
    ph_df['ph_sig_down'] = raw_df.apply(lambda x: is_sig_down(x), axis=1)
    # significant decrease
    ph_df['ph_sig_up'] = raw_df.apply(lambda x: is_sig_up(x), axis=1)
    # convenience column for filtering
    ph_df['ph_detected'] = 1
    #ph_df = ph_df.set_index('Uniprot_PTM')
    return ph_df
    
def make_ub_df(raw_df):
    ub_df = raw_df[['Gene', 'Protein','iLog2FC', 'iPvalue', 'Uniprot_PTM']]
    ub_df = ub_df.rename(columns={'Gene': 'mouse_gene', 
                                                'Protein': 'protein', 
                                                'iLog2FC': 'ub_ilog2fc',
                                                'iPvalue': 'ub_ipvalue',
                                                'Uniprot_PTM': 'ub_ptm'
                                               })
    ub_df['name'] = ub_df['mouse_gene'].str.upper()
    # significant decrease
    ub_df['ub_sig_down'] = raw_df.apply(lambda x: is_sig_down(x), axis=1)
    # significant decrease
    ub_df['ub_sig_up'] = raw_df.apply(lambda x: is_sig_up(x), axis=1)
    # convenience column for filtering
    ub_df['ub_detected'] = 1
    #ub_df = ub_df.set_index('Uniprot_PTM')
    return ub_df
# process raw dfs, load data into timepoint map

ab_df = make_ab_df(raw_ab_df)

ph_df = make_ph_df(raw_ph_df)

ub_df = make_ub_df(raw_ub_df)
is_duplicated = ab_df['protein'].duplicated()
duplicated_ids = ab_df['protein'][is_duplicated]
print(duplicated_ids)
ab_df = ab_df.set_index('protein')



Series([], Name: protein, dtype: object)


In [16]:
ph_df


,mouse_gene,protein,ph_ilog2fc,ph_ipvalue,ph_ptm,name,ph_sig_down,ph_sig_up,ph_detected
0,Tbc1d25,A1A5B6,-0.103224,0.782225,A1A5B6_S560,TBC1D25,0,0,1
1,Cul4b,A2A432,-2.032629,0.000721,A2A432_S104,CUL4B,1,0,1
2,Cul4b,A2A432,-0.651189,0.206512,A2A432_S149,CUL4B,0,0,1
3,Dhx8,A2A4P0,-0.039749,0.936598,A2A4P0_S480,DHX8,0,0,1
4,Dhx8,A2A4P0,-0.370576,0.043013,A2A4P0_S484,DHX8,0,0,1
...,...,...,...,...,...,...,...,...,...
6800,Elf4,Q9Z2U4,0.162078,0.717468,Q9Z2U4_S185,ELF4,0,0,1
6801,Elf4,Q9Z2U4,0.232996,0.392230,Q9Z2U4_S187,ELF4,0,0,1
6802,Q9Z2V6,Q9Z2V6,-0.103410,0.862226,Q9Z2V6_S650,Q9Z2V6,0,0,1
6803,Q9Z2V6,Q9Z2V6,0.276717,0.505649,Q9Z2V6_S650;Q9Z2V6_S660,Q9Z2V6,0,0,1


In [17]:
def add_null_values(tp_map):
    proteins = tp_map['proteins']
    for index, map_row in proteins.items():
        map_row['max_abs_log2fc'] = 0
        if not map_row.get('ab_detected'):
            add_null_ab_values(map_row)
        if not map_row.get('ub_detected'):
            add_null_ptm_values(map_row, 'ub')
        if not map_row.get('ph_detected'):
            add_null_ptm_values(map_row, 'ph')

def add_max_abs_log2fc(tp_map):
    proteins = tp_map['proteins']
    for index, map_row in proteins.items():
        map_row['max_abs_log2fc'] = max(abs(map_row.get('ab_ilog2fc')), 
                                        map_row.get('ph_max_ilog2fc'),
                                        abs(map_row.get('ph_min_ilog2fc')), 
                                        map_row.get('ub_max_ilog2fc'),
                                        abs(map_row.get('ub_min_ilog2fc')))
def add_null_ab_values(map_row):
    map_row['ab_ipvalue'] = 1
    map_row['ab_ilog2fc'] = 0
    map_row['ab_sig_up'] = 0
    map_row['ab_sig_down'] = 0
    map_row['ab_detected'] = 0

def add_null_ptm_values(map_row, ptm):
    map_row[ptm + '_max_ilog2fc'] = 0
    map_row[ptm + '_min_ilog2fc'] = 0
    map_row[ptm + '_sig_up'] = 0
    map_row[ptm + '_sig_down'] = 0
    map_row[ptm + '_detected'] = 0
    map_row[ptm + '_min_down_ipvalue'] = 1
    map_row[ptm + '_min_up_ipvalue'] = 1
    map_row[ptm + '_all_down_ptm'] = ''
    map_row[ptm + '_all_up_ptm'] = '' 
            
def add_new_ptm_values(map_row, row, ptm):
    # upregulation
    map_row[ptm + '_max_ilog2fc'] = row[ptm + '_ilog2fc']
    map_row[ptm + '_min_ilog2fc'] = row[ptm + '_ilog2fc']
    map_row[ptm + '_sig_up'] = row[ptm + '_sig_up']
    map_row[ptm + '_sig_down'] = row[ptm + '_sig_down']
    map_row[ptm + '_detected'] = row[ptm + '_detected']
    if row[ptm + '_ilog2fc'] > 0: 
            map_row[ptm + '_min_up_ipvalue'] = row[ptm + '_ipvalue']
            map_row[ptm + '_min_down_ipvalue'] = 1
            map_row[ptm + '_all_up_ptm'] = row[ptm + '_ptm']
            map_row[ptm + '_all_down_ptm'] = ''
    else:
            map_row[ptm + '_min_down_ipvalue'] = row[ptm + '_ipvalue']
            map_row[ptm + '_min_up_ipvalue'] = 1
            map_row[ptm + '_all_down_ptm'] = row[ptm + '_ptm']
            map_row[ptm + '_all_up_ptm'] = '' 
                
def merge_ptm_values(map_row, row, ptm):
    if row[ptm + '_ilog2fc'] > 0: 
            # upregulation
            map_row[ptm + '_min_up_ipvalue'] = min(map_row.get(ptm + '_min_up_ipvalue'), row[ptm + '_ipvalue'])
            map_row[ptm + '_all_up_ptm'] = row[ptm + '_ptm'] + map_row.get(ptm + '_all_up_ptm')
            map_row[ptm + '_sig_up'] = row[ptm + '_sig_up'] + map_row.get(ptm + '_sig_up')
            map_row[ptm + '_max_ilog2fc'] = max(map_row.get(ptm + '_max_ilog2fc'), row[ptm + '_ilog2fc'])
    else:
            # downregulation
            map_row[ptm + '_min_down_ipvalue'] = min(map_row.get(ptm + '_min_down_ipvalue'), row[ptm + '_ipvalue'])    
            map_row[ptm + '_all_down_ptm'] = row[ptm + '_ptm'] + map_row.get(ptm + '_all_down_ptm')
            map_row[ptm + '_sig_down'] = row[ptm + '_sig_down'] + map_row.get(ptm + '_sig_down')
            map_row[ptm + '_min_ilog2fc'] = min(map_row.get(ptm + '_min_ilog2fc'), row[ptm + '_ilog2fc'])    
    
def make_timepoint_map(timepoint, ab_df, ph_df, ub_df):
    tp_map = {'timepoint': timepoint, 'proteins': ab_df.to_dict('index')}
    # process the phospho and ubiquitylation site data
    proteins = tp_map['proteins']
    process_ptms(proteins, ph_df, 'ph')
    process_ptms(proteins, ub_df, 'ub')
    add_null_values(tp_map)
    add_max_abs_log2fc(tp_map)
    return tp_map

def process_ptms(proteins, ptm_df, ptm):   
    for index, row in ptm_df.iterrows():
        # for each row in ptm_df, 
        # if the protein is not already in tp_map
        protein = row['protein']
        map_row = proteins.get(protein)
        if not map_row:
            # make a dict the values from the row plus default values for the ab columns
            # then insert it into tp_map
            # TODO: the name ought to be the real protein name, minus 'MOUSE_', not the upcased mouse gene name.
            # this will mean that some proteins will not merge with the reference network
            map_row = {'mouse_gene': row['mouse_gene'], 'name': row['mouse_gene'].upper()}
            add_new_ptm_values(map_row, row, ptm)
            proteins[protein] = map_row
        else:
            # if the map_row exists but does not have ph values, add new ones
            if not map_row.get(ptm + '_detected'):
                add_new_ptm_values(map_row, row, ptm)
            else:
                merge_ptm_values(map_row, row, ptm) 
    # clean up so that all attribures have a valid, non-null value

    

tp_map = make_timepoint_map(timepoint, ab_df, ph_df, ub_df)
tp_map.items()

dict_items([('timepoint', '24H'), ('proteins', {'A0A0B4J1G0': {'mouse_gene': 'A0A0B4J1G0', 'ab_ilog2fc': 2.91607392272203, 'ab_ipvalue': 4.1482319170315e-08, 'name': 'A0A0B4J1G0', 'ab_sig_down': 0, 'ab_sig_up': 1, 'ab_detected': 1, 'max_abs_log2fc': 2.91607392272203, 'ub_max_ilog2fc': 0, 'ub_min_ilog2fc': 0, 'ub_sig_up': 0, 'ub_sig_down': 0, 'ub_detected': 0, 'ub_min_down_ipvalue': 1, 'ub_min_up_ipvalue': 1, 'ub_all_down_ptm': '', 'ub_all_up_ptm': '', 'ph_max_ilog2fc': 0, 'ph_min_ilog2fc': 0, 'ph_sig_up': 0, 'ph_sig_down': 0, 'ph_detected': 0, 'ph_min_down_ipvalue': 1, 'ph_min_up_ipvalue': 1, 'ph_all_down_ptm': '', 'ph_all_up_ptm': ''}, 'P58742': {'mouse_gene': 'Aaas', 'ab_ilog2fc': -1.08092371371379, 'ab_ipvalue': 0.414923104979028, 'name': 'AAAS', 'ab_sig_down': 0, 'ab_sig_up': 0, 'ab_detected': 1, 'max_abs_log2fc': 1.08092371371379, 'ub_max_ilog2fc': 0, 'ub_min_ilog2fc': 0, 'ub_sig_up': 0, 'ub_sig_down': 0, 'ub_detected': 0, 'ub_min_down_ipvalue': 1, 'ub_min_up_ipvalue': 1, 'ub_all_

In [18]:
tp_df = pd.DataFrame.from_dict(tp_map['proteins'], orient='index')
tp_df

,mouse_gene,ab_ilog2fc,ab_ipvalue,name,ab_sig_down,ab_sig_up,ab_detected,max_abs_log2fc,ub_max_ilog2fc,ub_min_ilog2fc,...,ub_all_up_ptm,ph_max_ilog2fc,ph_min_ilog2fc,ph_sig_up,ph_sig_down,ph_detected,ph_min_down_ipvalue,ph_min_up_ipvalue,ph_all_down_ptm,ph_all_up_ptm
A0A0B4J1G0,A0A0B4J1G0,2.916074,4.148232e-08,A0A0B4J1G0,0,1,1,2.916074,0.000000,0.000000,...,,0.000000,0.000000,0,0,0,1.00000,1.000000,,
P58742,Aaas,-1.080924,4.149231e-01,AAAS,0,0,1,1.080924,0.000000,0.000000,...,,0.000000,0.000000,0,0,0,1.00000,1.000000,,
Q9D2R0,Aacs,-0.445971,1.496781e-03,AACS,0,0,1,0.445971,0.000000,0.000000,...,,0.000000,0.000000,0,0,0,1.00000,1.000000,,
Q8R2R3,Aagab,0.166962,8.175600e-01,AAGAB,0,0,1,0.166962,0.000000,0.000000,...,,0.000000,0.000000,0,0,0,1.00000,1.000000,,
Q3UHJ0,Aak1,0.074436,8.186503e-01,AAK1,0,0,1,7.978963,0.000000,0.000000,...,,7.978963,-6.546346,2,2,1,0.00264,0.011402,Q3UHJ0_T671;Q3UHJ0_S674Q3UHJ0_T638Q3UHJ0_T618;...,Q3UHJ0_T618;Q3UHJ0_S622Q3UHJ0_T608;Q3UHJ0_T618...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Q9QXM0,Abhd2,0.000000,1.000000e+00,ABHD2,0,0,0,0.505617,0.505617,0.505617,...,Q9QXM0_K57,0.000000,0.000000,0,0,0,1.00000,1.000000,,
Q9QY73,Tmem59,0.000000,1.000000e+00,TMEM59,0,0,0,0.739210,-0.739210,-0.739210,...,,0.000000,0.000000,0,0,0,1.00000,1.000000,,
Q9WVM1,Racgap1,0.000000,1.000000e+00,RACGAP1,0,0,0,5.206364,-5.206364,-5.206364,...,,0.000000,0.000000,0,0,0,1.00000,1.000000,,
Q9Z0F5,Ch25h,0.000000,1.000000e+00,CH25H,0,0,0,2.702011,2.702011,2.702011,...,Q9Z0F5_K294,0.000000,0.000000,0,0,0,1.00000,1.000000,,


In [19]:
tp_df.to_excel("budzik_" + tp_map['timepoint'] + ".xlsx")